# Imports

In [1]:
import sys
import os

In [2]:
import math

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
import pandas as pd

In [6]:
import scipy as sc

In [7]:
import mat73

In [8]:
import mne

In [9]:
from pyvista import start_xvfb

## Config

In [10]:
start_xvfb()

In [11]:
%matplotlib qt

# Load Data

In [12]:
data = mat73.loadmat('data/osfstorage-archive/prfresults.mat')

In [13]:
data.keys()

dict_keys(['allresults', 'ciftifsaveragebad', 'ciftifsaverageix', 'groupsubjectids', 'quants', 'subjectids'])

- `allresults` - is 91282 grayordinates x 6 quantities x 184 datasets x 3 model fits with the full set of pRF analysis results.  
- `quants` - is a 1 x 6 cell vector with a label for the 6 quantities: {'ang' 'ecc' 'gain' 'meanvol' 'R2' 'rfsize'}.  
- `subjectids` - is 184 x 1 with the 6-digit ID for each subject (both individual and group-average subjects).  
- `ciftifsaverageix` - is 327684 x 1 with indices into the 91282 CIFTI space. This indexing vector performs a nearest-neighbor mapping from CIFTI to FreeSurfer's fsaverage space. Note that some of the fsaverage vertices do not have a counterpart in the CIFTI space; these vertices are assigned a value of 1 in 'ciftifsaverageix'.  
- `ciftifsaveragebad` - is 327684 x 1 with logical values indicating which fsaverage vertices do not have a counterpart in the CIFTI space. These vertices should receive a NaN after performing the indexing-based mapping.  
- `groupsubjectids` - is a 1 x 3 cell vector indicating which subjects contributed to the special 999997, 999998, and 999999 group-average subjects, respectively. Each element is a vector of indices (in the range 1-181) in sorted order.

In [14]:
data["allresults"].shape

(91282, 6, 184, 3)

91282 grayordinates x 6 quantities x 184 datasets x 3 model fits

In [15]:
data["quants"]

['ang', 'ecc', 'gain', 'meanvol', 'R2', 'rfsize']

- `ang` is the angle of the estimated pRF location. Specifically, it is the angle that the center of the pRF has with respect to the positive x-axis. The range of values is 0-360 and the units are degrees. Note that when the estimated pRF is exactly at the center of gaze (i.e., eccentricity is exactly 0), then angle is ill-defined. Thus, we have explicitly set the angle to NaN for any case in which eccentricity is 0.  
- `ecc` is the eccentricity of the estimated pRF location. Specifically, it is the distance from the center of gaze to the center of the pRF. Values are greater than or equal to 0 and the units are degrees of visual angle.  
- `gain` is the BOLD response amplitude estimated in the model fit. The units are raw scanner units (same units as the time-series data in the CIFTI files). The value can be interpreted as the amplitude of the BOLD response that is predicted to result from 1 s of full-field visual stimulation.  
- `meanvol` is the mean signal intensity observed in the time-series data. The units are raw scanner units (same units as the time-series data in the CIFTI files). Note that some grayordinates have a mean signal intensity that are zero or negative, so be careful (e.g., if converting to percent BOLD signal change).  
- `R2` is the amount of variance (R-squared) in the time-series data explained by the pRF model. The values generally range from 0-100 and the units are percentages. Note that R-squared values are computed after projecting out low-order polynomials from both the time-series data and the model fit. Because of this step, R-squared values can sometimes drop below 0%.  
- `rfsize` is the size of the estimated pRF. Specifically, it is one standard deviation of a 2D Gaussian that describes the behavior of the grayordinate (see paper for details). Values are positive and the units are degrees of visual angle.

In [16]:
# Taking the results corresponding to the pRF model fit based on all data of subject 999999
df = pd.DataFrame(data["allresults"][:, :, np.argwhere(data['subjectids'] == 999999).squeeze(), 0], columns=data['quants'])
# df.head()

In [17]:
atlas = mat73.loadmat('data/osfstorage-archive/atlas.mat')

In [18]:
print(atlas['wang2015labels'])

['Unknown', 'V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1', 'FEF']


In [19]:
df['label'] = np.array(atlas['wang2015labels'])[atlas['wang2015'].astype(int)]

In [20]:
df.head()

,ang,ecc,gain,meanvol,R2,rfsize,label
0,17.194132,4.696335,1.690906,11303.163086,10.583216,1.860162,Unknown
1,326.100861,5.211594,0.634324,12108.946289,4.457683,1.543650,Unknown
2,11.250000,5.568000,0.438437,11821.662109,2.194166,1.810193,Unknown
3,11.250000,5.568000,1.775627,12052.477539,11.634434,10.240000,Unknown
4,333.782410,6.588593,21.910872,12125.067383,63.411598,1.882475,V3A


# Main

In [21]:
dfv = df[df['label'].isin(('V1v', 'V1d'))]

In [22]:
dfv

,ang,ecc,gain,meanvol,R2,rfsize,label
21352,282.324371,3.671535,52.931629,11617.826172,88.788147,0.676684,V1d
21353,280.628357,3.348240,47.151394,11649.219727,84.843956,0.478202,V1d
21354,289.487396,3.324515,55.897320,11800.312500,88.069824,0.452826,V1d
21355,302.966858,3.241618,56.459423,12120.927734,86.946693,0.504809,V1d
21356,313.990112,3.117386,65.421722,12586.037109,85.429909,0.322909,V1d
...,...,...,...,...,...,...,...
53189,292.504730,8.009211,3.703999,11501.517578,10.685963,0.060616,V1d
53190,23.399443,7.964127,3.469547,11494.819336,8.177179,0.055189,V1d
53231,146.051498,7.940330,4.313904,11519.063477,17.152845,0.032037,V1d
53233,33.750000,13.656000,413.173035,11476.839844,5.753297,1.810193,V1d
